In [93]:
### Benjamin Tollison ###
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sympy as sp
from IPython.display import Latex, Math, display
from sympy import (
    Eq,
    Function,
    Matrix,
    cos,
    cosh,
    exp,
    integrate,
    lambdify,
    pi,
    sin,
    sinh,
    symbols,
)
from decimal import Decimal
from sympy.solvers.pde import pdsolve
from sympy.solvers.solveset import linsolve
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)
def displayVariable(variable:str,RHS):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
def displayVariableWithUnits(variable:str,RHS,units):
    left = sp.latex(symbols(variable))
    right = sp.latex(RHS)
    latexUnit = sp.latex(symbols(units))
    display(Math(left + '=' + right + '\\;' +'\\left['+ latexUnit + '\\right]'))
def format_scientific(number:float):
    a = '%E' % number
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]
deg2rad = np.pi/180
rad2deg = 180/np.pi

### Setting the Globals based off of my information

In [94]:
astro_constants = pd.read_csv('AstroConstants.csv')
ephemeris = pd.read_csv('EphemerisData.csv')
x = 20
y = 4
earth_parking_radius = 10*x + 450 + astro_constants['Earth'][1]   # km
mars_periapsis_radius = 15*y +150 + astro_constants['Mars'][1]
mars_apoapsis_radius = 275*y + 2500 + astro_constants['Mars'][1]
def GetIndexFromList(list, target_value):
    for key, value in enumerate(list):
        if value == target_value:
            return key
departure_index = GetIndexFromList(ephemeris['Epoch'],'10-Nov-26')
departure_data = {key:value[departure_index] for key,value in ephemeris.items()}
mu_sun = astro_constants['Sun'][0]
mu_earth = astro_constants['Earth'][0]
mu_mars = astro_constants['Mars'][0]
unit_length_conversion = astro_constants['Earth'][3]
unit_time_conversion = np.sqrt(unit_length_conversion**3/mu_sun) # sec
print(unit_length_conversion,'km')
print(unit_time_conversion,'seconds')
print(unit_time_conversion/(60*60*24),'days')
print(mu_sun,unit_length_conversion**3/unit_time_conversion**2)
display(pd.DataFrame(ephemeris))
display(pd.DataFrame(departure_data,index=[departure_index]))
print(departure_data['Earth X'])

149597870.7 km
5022642.890913032 seconds
58.132440867048985 days
132712440041.9393 132712440041.93929


,Epoch,Earth X,Earth Y,Earth Z,Earth Xdot,Earth Ydot,Earth Zdot,Mars X,Mars Y,Mars Z,Mars Xdot,Mars Ydot,Mars Zdot
0,1-Apr-26,-1.467589e+08,-2.596065e+07,-11252418.52,5.156136,-26.928642,-11.671947,193782670.7,-63169669.47,-34201020.13,9.320738,22.638956,10.132573
1,2-Apr-26,-1.462918e+08,-2.828332e+07,-12259161.90,5.656401,-26.835687,-11.631776,194577093.9,-61210155.93,-33323663.26,9.068525,22.719727,10.176423
2,3-Apr-26,-1.457815e+08,-3.059764e+07,-13262298.92,6.154717,-26.735375,-11.588464,195349683.9,-59243774.59,-32442567.40,8.815347,22.797931,10.219122
3,4-Apr-26,-1.452283e+08,-3.290298e+07,-14261557.55,6.651050,-26.627694,-11.541995,196100359.1,-57270747.61,-31557832.24,8.561242,22.873559,10.260665
4,5-Apr-26,-1.446323e+08,-3.519869e+07,-15256664.17,7.145344,-26.512620,-11.492349,196829040.8,-55291297.98,-30669557.95,8.306246,22.946602,10.301046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386,16-Jan-30,-6.328359e+07,1.218936e+08,52837760.35,-27.366695,-11.858771,-5.139106,201863133.8,-39278038.74,-23459454.69,6.235094,23.436334,10.581640
1387,17-Jan-30,-6.563809e+07,1.208502e+08,52385597.19,-27.134269,-12.293109,-5.327355,202390561.9,-37250983.41,-22543907.75,5.973802,23.485802,10.611376
1388,18-Jan-30,-6.797217e+07,1.197695e+08,51917241.86,-26.894022,-12.723439,-5.513923,202895391.9,-35219768.54,-21625843.54,5.711994,23.532619,10.639910
1389,19-Jan-30,-7.028516e+07,1.186517e+08,51432840.00,-26.646065,-13.149733,-5.698801,203377580.7,-33184623.24,-20705366.00,5.449707,23.576783,10.667240


,Epoch,Earth X,Earth Y,Earth Z,Earth Xdot,Earth Ydot,Earth Zdot,Mars X,Mars Y,Mars Z,Mars Xdot,Mars Ydot,Mars Zdot
223,10-Nov-26,100622923.6,99795464.56,43259105.48,-22.360161,18.463842,8.003854,-60800396.04,210334752.7,98115468.4,-22.520655,-3.932676,-1.196445


100622923.6


In [95]:
def Alpha(a,s):
  return  2*np.arcsin(np.sqrt(s/(2*a)))
def alphaPrime(a,s):
  x_squared = s/(2*a)
  dx = (-1/2)*np.sqrt(s/(2*a**3))
  return dx/np.sqrt(1-x_squared)
def Beta(a,s,c):
  return 2*np.arcsin(np.sqrt((s-c)/(2*a)))
def betaPrime(a,s,c):
  x_squared = (s-c)/(2*a)
  dx = (-1/2)*np.sqrt((s-c)/(2*a**3))
  return dx/np.sqrt(1-x_squared)
def P_transfer(a,s,r1,r2,c,alphabeta):
  p = ((4*a*(s-r1)*(s-r2))/c**2)*np.sin((alphabeta)/2)
  return p
def LambertFunction(a,alpha,beta,non_dimensional_TOF):
  return np.sqrt(a**3)*(alpha-beta -(np.sin(alpha)-np.sin(beta))) - non_dimensional_TOF
def LambertFunctionPrime(a,alpha,alphaprime,beta,betaprime):
  return (3/2)*np.sqrt(a)*(alphaprime-betaprime -(alphaprime*np.cos(alpha)-betaprime*np.cos(beta))) 
def OneATransfer(a0,s,c,non_dimensional_TOF):
  alpha = Alpha(a0,s)
  beta = Beta(a0,s,c)
  alphaprime = alphaPrime(a0,s)
  betaprime = betaPrime(a0,s,c)
  iteration_max = 1000
  while iteration_max !=0:
    lambert = LambertFunction(a0,alpha,beta,non_dimensional_TOF)
    lambertprime = LambertFunctionPrime(a0,alpha,alphaprime,beta,betaprime)
    # print(lambert)
    if abs(lambert) < 1e-4:
      # print(a0)
      return a0
    elif iteration_max == 0:
      return print('method didnt converge')
    else:
      a0 = a0 + lambert/lambertprime
      iteration_max -= 1
      # print(iteration_max)
def OneBTransfer(a0,s,c,non_dimensional_TOF):
  alpha = 2*np.pi - Alpha(a0,s)
  beta = Beta(a0,s,c)
  alphaprime = alphaPrime(a0,s)
  betaprime = betaPrime(a0,s,c)
  iteration_max = 1000
  while iteration_max !=0:
    lambert = LambertFunction(a0,alpha,beta,non_dimensional_TOF)
    lambertprime = LambertFunctionPrime(a0,alpha,alphaprime,beta,betaprime)
    # print(lambert)
    if abs(lambert) < 1e-4:
      # print(a0)
      return a0
    elif iteration_max == 0:
      return print('method didnt converge')
    else:
      a0 = a0 + lambert/lambertprime
      iteration_max -= 1
      # print(iteration_max)
def TwoATransfer(a0,s,c,non_dimensional_TOF):
  alpha = Alpha(a0,s)
  beta = -Beta(a0,s,c)
  alphaprime = alphaPrime(a0,s)
  betaprime = betaPrime(a0,s,c)
  iteration_max = 1000
  while iteration_max !=0:
    lambert = LambertFunction(a0,alpha,beta,non_dimensional_TOF)
    lambertprime = LambertFunctionPrime(a0,alpha,alphaprime,beta,betaprime)
    # print(lambert)
    if abs(lambert) < 1e-4:
      # print(f'a_2a={a0}')
      return a0
    elif iteration_max == 0:
      return print('method didnt converge')
    else:
      a0 = a0 + lambert/lambertprime
      iteration_max -= 1
      # print(iteration_max)
def TwoBTransfer(a0,s,c,non_dimensional_TOF):
  alpha = 2*np.pi - Alpha(a0,s)
  beta = -Beta(a0,s,c)
  alphaprime = alphaPrime(a0,s)
  betaprime = betaPrime(a0,s,c)
  iteration_max = 1000
  while iteration_max !=0:
    lambert = LambertFunction(a0,alpha,beta,non_dimensional_TOF)
    lambertprime = LambertFunctionPrime(a0,alpha,alphaprime,beta,betaprime)
    # print(lambert)
    if abs(lambert) < 1e-4:
      # print(a0)
      return a0
    elif iteration_max == 0:
      return print('method didnt converge')
    else:
      a0 = a0 + lambert/lambertprime
      iteration_max -= 1
      # print(iteration_max)
  return print('TODO 2B')
def EllipticTransfer(s,c,non_dimensional_TOF,TOF_p1,TOF_p2):
  a_min = 0.5*s
  alpha_min = Alpha(a_min,s)
  beta_min = Beta(a_min,s,c)
  TOFm1 = np.sqrt(a_min**3/1)*((alpha_min - np.sin(alpha_min)) - (beta_min-np.sin(beta_min)))
  TOFm2 = np.sqrt(a_min**3/1)*((alpha_min - np.sin(alpha_min)) + (beta_min-np.sin(beta_min)))
  a0 = a_min + 1e-4
  a_transfers = [0,0,0,0]
  if TOF_p1 < non_dimensional_TOF and non_dimensional_TOF < TOFm1:
    a_t1a = OneATransfer(a0,s,c,non_dimensional_TOF)
    a_transfers[0] = a_t1a
  if TOFm1 < non_dimensional_TOF:
    a_t1b = OneBTransfer(a0,s,c,non_dimensional_TOF)
    a_transfers[1] = a_t1b
  if TOF_p2 < non_dimensional_TOF and non_dimensional_TOF < TOFm2:
    a_t2a = TwoATransfer(a0,s,c,non_dimensional_TOF)
    a_transfers[2] = a_t2a
  if TOFm2 < non_dimensional_TOF:
    a_t2b = TwoBTransfer(a0,s,c,non_dimensional_TOF)
    a_transfers[3] = a_t2b
  return a_transfers
def HyperbolicTransfer():
  return print('TODO Hyperbolic')
def F(r2,p,delta_f):
  return 1 - (r2/p)*(1-np.cos(delta_f))
def G(r1,r2,p,delta_f):
  return ((r1*r2)/np.sqrt(p))*np.sin(delta_f)
def G_dot(r1,p,delta_f):
  return 1 - (r1/p)*(1-np.cos(delta_f))
def F_dot(r1,r2,p,delta_f):
  return (F(r2,p,delta_f)*G_dot(r1,p,delta_f)-1)/G(r1,r2,p,delta_f)

In [98]:
def LamberSolverA(TOF):
  arrival_index = departure_index + TOF
  arrival_data = {key:value[arrival_index] for key,value in ephemeris.items()}
  r1_vec = np.array([departure_data['Earth X'],departure_data['Earth Y'],departure_data['Earth Z']])/unit_length_conversion
  r2_vec = np.array([arrival_data['Earth X'],arrival_data['Earth Y'],arrival_data['Earth Z']]) /unit_length_conversion
  c_vec = r2_vec - r1_vec
  v_earth = np.array([departure_data['Earth Xdot'],departure_data['Earth Ydot'],departure_data['Earth Zdot']])
  v_mars = np.array([arrival_data['Mars Xdot'],arrival_data['Mars Ydot'],arrival_data['Mars Zdot']])
  print(v_mars)
  r1,r2,c = np.linalg.norm(r1_vec),np.linalg.norm(r2_vec),np.linalg.norm(c_vec)
  s = 0.5*(r1+r2+c)
  non_dimensional_TOF = (TOF*24*3600)/unit_time_conversion
  TOF_p1 = (1/3)*np.sqrt(2/1)*(s**(3/2) - (s-c)**(3/2))
  TOF_p2 = (1/3)*np.sqrt(2/1)*(s**(3/2) + (s-c)**(3/2))
  if non_dimensional_TOF < TOF_p1 or non_dimensional_TOF <= TOF_p2:
    HyperbolicTransfer()
  if non_dimensional_TOF > TOF_p2:
    transfer_velocities = {'v1':[],'v2':[]}
    p_t = []
    J = []
    display(EllipticTransfer(s,c,non_dimensional_TOF,TOF_p1,TOF_p2))
    index = 0
    for a_t in EllipticTransfer(s,c,non_dimensional_TOF,TOF_p1,TOF_p2):
      if a_t != 0:
        alphabeta = Alpha(a_t,s) + Beta(a_t,s,c)*(-1)**(index+2)
        p_t.append(P_transfer(a_t,s,r1,r2,c,alphabeta))
        p = P_transfer(a_t,s,r1,r2,c,alphabeta)
        if index <= 1:
          deltaf = np.arccos(np.dot(r1_vec,r2_vec)/(r1*r2))
          v1t = (1/G(r1,r2,p,deltaf))*(r2_vec-F(r2,p,deltaf)*r1_vec)
          v2t = F_dot(r1,r2,p,deltaf)*r1_vec + G_dot(r1,p,deltaf)*v1t
          transfer_velocities['v1'].append(v1t),transfer_velocities['v2'].append(v2t)
        else:
          deltaf = np.pi - np.arccos(np.dot(r1_vec,r2_vec)/(r1*r2))
          v1t = (1/G(r1,r2,p,deltaf))*(r2_vec-F(r2,p,deltaf)*r1_vec)
          v2t = F_dot(r1,r2,p,deltaf)*r1_vec + G_dot(r1,p,deltaf)*v1t
          transfer_velocities['v1'].append(v1t),transfer_velocities['v2'].append(v2t)
      else:
        p_t.append(0.0)
      index +=1
    if not np.isnan(transfer_velocities['v1'][0][0]):
      for i in range(2):
        J_i = (np.linalg.norm(transfer_velocities['v1'][i]-v_earth)) + (np.linalg.norm(transfer_velocities['v2'][i]-v_mars))
        J.append(J_i)
      print(J) 
    else:
      print("The value is NaN.")
      return None
    return print('TODO LamberSolverA')
for days in range(100,701):
  LamberSolverA(days)

[-11.33014309 -17.24157556  -7.60280041]


[0.6847635152839586, 0, 0.6664649808575551, 0]

The value is NaN.
[-11.17037639 -17.32427381  -7.64504179]


C:\Users\benja\AppData\Local\Temp\ipykernel_20592\216515559.py:2: RuntimeWarning: invalid value encountered in arcsin
  return  2*np.arcsin(np.sqrt(s/(2*a)))


[0.6889086229746878, 0, 0.6712664108898311, 0]

...

[55.79096060470418, 119.49510150349076]
TODO LamberSolverA
[25.08817794  3.61224441  0.98030467]


[0, 2.5707892088482973, 0, 1.161941477505053]

[55.788899262414404, 134.28447456447302]
TODO LamberSolverA
[25.07061721  3.83391439  1.08245414]


[0, None, 0, 1.1515637242254242]

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'